In [ ]:
# Import necessary libraries
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [ ]:
# Load the dataset
text_df = pd.read_csv("/content/fake_or_real_news.csv")

In [ ]:
# Combine text data into a single string
text = list(text_df.text.values)
joined_text = " ".join(text)

In [ ]:
# Take a partial text from the beginning
partial_text = joined_text[:10000]

In [ ]:
# Tokenize the text using regular expression
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [ ]:
# Find unique tokens and create a dictionary for mapping tokens to indices
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [ ]:
# Define the number of words for the input sequence
n_words = 10
input_words = []
next_words = []

In [ ]:
# Create input and output sequences for the model
for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])

In [ ]:
# Initialize the input and output arrays
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [ ]:
# Convert input and output sequences into a suitable format for the model
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [ ]:
# RNN Model
# Create a sequential model
rnn_model = Sequential()

In [ ]:
# Add a simple RNN layer with 128 units
rnn_model.add(SimpleRNN(128, input_shape=(n_words, len(unique_tokens),), return_sequences=True))
rnn_model.add(SimpleRNN(128))

In [ ]:
# Add a dense layer with the number of unique tokens and a softmax activation function
rnn_model.add(Dense(len(unique_tokens)))
rnn_model.add(Activation('softmax'))

In [ ]:
# Compile the RNN model with appropriate loss function, optimizer, and metrics
rnn_model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])

In [ ]:
# Train the RNN model using the training data
rnn_model.fit(X, y, batch_size=128, epochs=30, shuffle=True)

Epoch 1/30
14/14 [==============================] - 2s 34ms/step - loss: 6.9907 - accuracy: 0.0166
Epoch 2/30
14/14 [==============================] - 0s 33ms/step - loss: 6.5228 - accuracy: 0.0246
Epoch 3/30
14/14 [==============================] - 0s 33ms/step - loss: 6.4462 - accuracy: 0.0338
Epoch 4/30
14/14 [==============================] - 0s 33ms/step - loss: 6.4622 - accuracy: 0.0309
Epoch 5/30
14/14 [==============================] - 0s 33ms/step - loss: 6.4027 - accuracy: 0.0320
Epoch 6/30
14/14 [==============================] - 0s 33ms/step - loss: 6.7954 - accuracy: 0.0252
Epoch 7/30
14/14 [==============================] - 0s 33ms/step - loss: 6.4434 - accuracy: 0.0349
Epoch 8/30
14/14 [==============================] - 0s 34ms/step - loss: 6.4406 - accuracy: 0.0338
Epoch 9/30
14/14 [==============================] - 0s 32ms/step - loss: 6.4079 - accuracy: 0.0269
Epoch 10/30
14/14 [==============================] - 0s 34ms/step - loss: 6.5194 - accuracy: 0.0229
Epoch 11/

In [ ]:
# Save the RNN model
rnn_model.save("mymodel_rnn.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# LSTM Model
# Create a sequential model
lstm_model = Sequential()

In [ ]:
# Add an LSTM layer with 128 units
lstm_model.add(LSTM(128, input_shape=(n_words, len(unique_tokens)), return_sequences=True))
lstm_model.add(LSTM(128))

In [ ]:
# Add a dense layer with the number of unique tokens and a softmax activation function
lstm_model.add(Dense(len(unique_tokens)))
lstm_model.add(Activation('softmax'))

In [ ]:
# Compile the LSTM model with appropriate loss function, optimizer, and metrics
lstm_model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=["accuracy"])

In [ ]:
# Train the LSTM model using the training data
lstm_model.fit(X, y, batch_size=128, epochs=30, shuffle=True)

Epoch 1/30
14/14 [==============================] - 5s 119ms/step - loss: 6.2298 - accuracy: 0.0366
Epoch 2/30
14/14 [==============================] - 2s 135ms/step - loss: 5.8606 - accuracy: 0.0618
Epoch 3/30
14/14 [==============================] - 1s 85ms/step - loss: 5.8086 - accuracy: 0.0618
Epoch 4/30
14/14 [==============================] - 1s 88ms/step - loss: 5.7877 - accuracy: 0.0618
Epoch 5/30
14/14 [==============================] - 1s 87ms/step - loss: 5.7531 - accuracy: 0.0618
Epoch 6/30
14/14 [==============================] - 1s 87ms/step - loss: 5.6958 - accuracy: 0.0618
Epoch 7/30
14/14 [==============================] - 1s 87ms/step - loss: 5.6402 - accuracy: 0.0618
Epoch 8/30
14/14 [==============================] - 1s 86ms/step - loss: 5.5644 - accuracy: 0.0601
Epoch 9/30
14/14 [==============================] - 1s 87ms/step - loss: 5.4086 - accuracy: 0.0761
Epoch 10/30
14/14 [==============================] - 1s 87ms/step - loss: 5.2418 - accuracy: 0.0830
Epoch 1

In [ ]:
# Save the LSTM model
lstm_model.save("mymodel_lstm.h5")

In [ ]:
# Function to predict the next word based on the input text and the trained model
def predict_next_word(model, input_text, n_best, unique_tokens, unique_token_index, n_words):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for i, word in enumerate(input_text.split()):
        X[0, i, unique_token_index[word]] = 1

    predictions = model.predict(X)[0]
    return np.argpartition(predictions, -n_best)[-n_best:]

In [ ]:
# Function to generate text based on the input text and the trained model
def generate_text(model, input_text, text_length, creativity, unique_tokens, unique_token_index, n_words):
    word_sequence = input_text.split()
    current = 0
    for _ in range(text_length):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current + n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(model, sub_sequence, creativity, unique_tokens, unique_token_index, n_words))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [ ]:
# Generate text using the RNN model
generated_text_rnn = generate_text(rnn_model, "He will have to look into this thing and he", 100, 5, unique_tokens, unique_token_index, n_words)

1/1 [==============================] - 0s 20ms/step


In [ ]:
# Generate text using the LSTM model
generated_text_lstm = generate_text(lstm_model, "He will have to look into this thing and he", 100, 5, unique_tokens, unique_token_index, n_words)

1/1 [==============================] - 0s 25ms/step


In [ ]:
# Print the generated texts RNN
print("Generated Text (RNN): ", generated_text_rnn)

Generated Text (RNN):  He will have to look into this thing and he are a are of the a are the fighting the of fighting fighting a fighting the are a the a the the fighting the fighting of a the the of the of of fighting are of of of fighting a a are the fighting the are of are a of the of fighting fighting the the the are are of a a a the of a the fighting fighting a fighting of a the the fighting of fighting of of the a of of fighting a a are the of are fighting of a are the are fighting a a


In [ ]:
# Print the generated texts LSTM
print("Generated Text (LSTM): ", generated_text_lstm)

Generated Text (LSTM):  He will have to look into this thing and he defending he lie and isn it his be him for anthony and s paul to be him between way has chances to be himself the time and obama as the final stretch of her ever vytt49yvoe campaign going near is a afraid for s compared hillary it the chances of its bigger that it just up be for stand the people has done is comey and during it front election she ought that has changed hillary desperation like before reason the same this of especially it s gone what her the revelation now the old candidate then the years is
